In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# Define task ratings files used for each year
onet_task_ratings_dir = '/Users/sidsatya/dev/ailabor/data/onet/historical_onet_task_ratings'

# TODO: add in 2003 - 2007
onet_task_files = {
    2008: os.path.join(onet_task_ratings_dir, "task_ratings_2008_jun.csv"),  # Follows SOC 2006
    2009: os.path.join(onet_task_ratings_dir, "task_ratings_2009_jun.csv"),  # Follows SOC 2006
    2010: os.path.join(onet_task_ratings_dir, "task_ratings_2010_jul.csv"),  # Follows SOC 2009
    2011: os.path.join(onet_task_ratings_dir, "task_ratings_2011_jul.csv"),  # Follows SOC 2009
    2012: os.path.join(onet_task_ratings_dir, "task_ratings_2012_jul.csv"),  # Follows SOC 2010
    2013: os.path.join(onet_task_ratings_dir, "task_ratings_2013_jul.csv"),  # Follows SOC 2010
    2014: os.path.join(onet_task_ratings_dir, "task_ratings_2014_jul.csv"),  # Follows SOC 2010
    2015: os.path.join(onet_task_ratings_dir, "task_ratings_2015_oct.csv"),  # Follows SOC 2010
    2016: os.path.join(onet_task_ratings_dir, "task_ratings_2016_nov.csv"),  # Follows SOC 2010
    2017: os.path.join(onet_task_ratings_dir, "task_ratings_2017_oct.csv"),  # Follows SOC 2010
    2018: os.path.join(onet_task_ratings_dir, "task_ratings_2018_nov.csv"),  # Follows SOC 2010
    2019: os.path.join(onet_task_ratings_dir, "task_ratings_2019_nov.csv"),  # Follows SOC 2010
    2020: os.path.join(onet_task_ratings_dir, "task_ratings_2020_nov.csv"),  # Follows SOC 2019
    2021: os.path.join(onet_task_ratings_dir, "task_ratings_2021_nov.csv"),  # Follows SOC 2019
    2022: os.path.join(onet_task_ratings_dir, "task_ratings_2022_nov.csv"),  # Follows SOC 2019
    2023: os.path.join(onet_task_ratings_dir, "task_ratings_2023_nov.csv"),  # Follows SOC 2019
    2024: os.path.join(onet_task_ratings_dir, "task_ratings_2024_nov.csv"),  # Follows SOC 2019
    2025: os.path.join(onet_task_ratings_dir, "task_ratings_2025_feb.csv"),  # Follows SOC 2019
}


In [ ]:
# read in data from ../data/onet/historical_onet_task_ratings. Create giant dataframe with all the data
def read_onet_task_data(onet_task_files):
    all_files = pd.DataFrame()
    for year, file_path in onet_task_files.items():
        df = pd.read_csv(file_path, encoding='latin1')
        df['year'] = year  # Use the key as the year value
        all_files = pd.concat([all_files, df], ignore_index=True)
    return all_files

task_ratings_2008_onwards = read_onet_task_data(onet_task_files)

In [21]:
'''
Clean up task ratings by only keeping rows with: 
1) Recommend Suppress = N
2) Scale ID in ('IM', 'FT'). This is subject to change if we want to add in more interesting attributes such as levels. 
'''

task_ratings_2008_onwards = task_ratings_2008_onwards[
    (task_ratings_2008_onwards['Recommend Suppress'] == 'N') &
    (task_ratings_2008_onwards['Scale ID'].isin(['IM', 'FT']))
]

print(len(task_ratings_2008_onwards), "rows after filtering for 'Recommend Suppress' and 'Scale ID'.")

1980529 rows after filtering for 'Recommend Suppress' and 'Scale ID'.


In [69]:
tr_2008_onwards_IM = task_ratings_2008_onwards[task_ratings_2008_onwards['Scale ID'] == 'IM'].copy()

# Calculate normalized IM score for each task within an occ-year group
tr_2008_onwards_IM_grp = tr_2008_onwards_IM.groupby(['O*NET-SOC Code', 'year']).agg(
    Data_Value_sum_all=('Data Value', 'sum')
).reset_index()

# merge with task_ratings_2008_onwards to get the task statements
tr_2008_onwards_IM = pd.merge(tr_2008_onwards_IM, tr_2008_onwards_IM_grp, on=['O*NET-SOC Code', 'year'], suffixes=('', '_total'))
tr_2008_onwards_IM['IM_normalized'] = tr_2008_onwards_IM['Data Value'] / tr_2008_onwards_IM['Data_Value_sum_all']

tr_2008_onwards_IM

,O*NET-SOC Code,Task ID,Scale ID,Category,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Date,Domain Source,year,Title,Task,Scale Name,Data_Value_sum_all,IM_normalized
0,11-1011.00,8823,IM,NaN,4.51,93.0,0.1300,4.2600,4.7700,N,06/2006,Incumbent,2008,NaN,NaN,NaN,120.26,0.037502
1,11-1011.00,8824,IM,NaN,4.38,102.0,0.1200,4.1500,4.6100,N,06/2006,Incumbent,2008,NaN,NaN,NaN,120.26,0.036421
2,11-1011.00,8825,IM,NaN,4.34,96.0,0.1500,4.0300,4.6400,N,06/2006,Incumbent,2008,NaN,NaN,NaN,120.26,0.036088
3,11-1011.00,8826,IM,NaN,4.19,98.0,0.1400,3.9100,4.4700,N,06/2006,Incumbent,2008,NaN,NaN,NaN,120.26,0.034841
4,11-1011.00,8827,IM,NaN,4.13,96.0,0.1400,3.8400,4.4200,N,06/2006,Incumbent,2008,NaN,NaN,NaN,120.26,0.034342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250530,53-7121.00,12807,IM,NaN,4.08,53.0,0.4084,3.2562,4.8952,N,08/2019,Incumbent,2025,"Tank Car, Truck, and Ship Loaders",Unload cars containing liquids by connecting h...,Importance,81.47,0.050080
250531,53-7121.00,12804,IM,NaN,4.02,28.0,0.1538,3.7006,4.3319,N,08/2019,Incumbent,2025,"Tank Car, Truck, and Ship Loaders","Clean interiors of tank cars or tank trucks, u...",Importance,81.47,0.049343
250532,53-7121.00,12803,IM,NaN,3.88,56.0,0.2332,3.4129,4.3475,N,08/2019,Incumbent,2025,"Tank Car, Truck, and Ship Loaders",Lower gauge rods into tanks or read meters to ...,Importance,81.47,0.047625
250533,53-7121.00,12805,IM,NaN,3.87,31.0,0.2604,3.3420,4.4058,N,08/2019,Incumbent,2025,"Tank Car, Truck, and Ship Loaders",Operate conveyors and equipment to transfer gr...,Importance,81.47,0.047502


In [70]:
# calculate Frequency score for each task within an occ-year group
tr_2008_onwards_FT = task_ratings_2008_onwards[task_ratings_2008_onwards['Scale ID'] == 'FT'].copy()
tr_2008_onwards_FT['weighted_frequency'] = tr_2008_onwards_FT['Category'] * tr_2008_onwards_FT['Data Value']

tr_2008_onwards_FT_grp = tr_2008_onwards_FT.groupby(['O*NET-SOC Code', 'Task ID', 'year', 'Scale ID']).agg({'weighted_frequency': 'sum'}).reset_index()
tr_2008_onwards_FT_grp['expected_freq'] = tr_2008_onwards_FT_grp['weighted_frequency']/100

tr_2008_onwards_FT_grp

,O*NET-SOC Code,Task ID,year,Scale ID,weighted_frequency,expected_freq
0,11-1011.00,8823,2008,FT,363.69,3.6369
1,11-1011.00,8823,2009,FT,363.69,3.6369
2,11-1011.00,8823,2010,FT,363.69,3.6369
3,11-1011.00,8823,2011,FT,363.69,3.6369
4,11-1011.00,8823,2012,FT,363.69,3.6369
...,...,...,...,...,...,...
250287,53-7121.00,12810,2021,FT,456.33,4.5633
250288,53-7121.00,12810,2022,FT,456.33,4.5633
250289,53-7121.00,12810,2023,FT,456.33,4.5633
250290,53-7121.00,12810,2024,FT,456.33,4.5633


In [73]:
# Merge attributes back in
tr_2008_onwards_merged = pd.merge(task_ratings_2008_onwards, tr_2008_onwards_IM[['O*NET-SOC Code', 'year', 'Task ID', 'Data Value', 'IM_normalized']],
                                      on=['O*NET-SOC Code', 'year', 'Task ID'], how='left')
tr_2008_onwards_merged = pd.merge(tr_2008_onwards_merged, tr_2008_onwards_FT_grp[['O*NET-SOC Code', 'year', 'Task ID', 'expected_freq']],
                                      on=['O*NET-SOC Code', 'year', 'Task ID'], how='left')

tr_2008_onwards_final = tr_2008_onwards_merged.groupby(['O*NET-SOC Code', 'Task ID', 'year', 'Date']).agg({ 'Data Value_y': 'mean',
                                                                                                            'IM_normalized': ['mean', lambda x: len(np.unique(x))], 
                                                                                                            'expected_freq': ['mean', lambda x: len(np.unique(x))]}).reset_index()

In [85]:
tr_2008_onwards_final.columns = ['O*NET-SOC Code', 'Task ID', 'year', 'Date', 'Mean Importance', 'Importance Normalized All', 'Importance Normalized unique', 'Mean Frequency', 'Mean Frequency unique']
tr_2008_onwards_final

,O*NET-SOC Code,Task ID,year,Date,Mean Importance,Importance Normalized All,Importance Normalized unique,Mean Frequency,Mean Frequency unique
0,11-1011.00,8823,2008,06/2006,4.51,0.037502,1,3.6369,1
1,11-1011.00,8823,2009,06/2006,4.51,0.037502,1,3.6369,1
2,11-1011.00,8823,2010,06/2006,4.51,0.037502,1,3.6369,1
3,11-1011.00,8823,2011,06/2006,4.51,0.037502,1,3.6369,1
4,11-1011.00,8823,2012,06/2006,4.51,0.037502,1,3.6369,1
...,...,...,...,...,...,...,...,...,...
250589,53-7121.00,12810,2021,08/2019,3.53,0.043329,1,4.5633,1
250590,53-7121.00,12810,2022,08/2019,3.53,0.043329,1,4.5633,1
250591,53-7121.00,12810,2023,08/2019,3.53,0.043329,1,4.5633,1
250592,53-7121.00,12810,2024,08/2019,3.53,0.043329,1,4.5633,1


In [57]:
''' Now we will deal with task ratings from 2004 to 2007, which are contained in the task statement files. Will have to treat 2003 differently.'''
onet_task_statements_dir = '/Users/sidsatya/dev/ailabor/data/onet/historical_onet_task_statements'
onet_task_files = {  
    2003: os.path.join(onet_task_statements_dir, "task_statements_2003_nov.csv"),  
    2004: os.path.join(onet_task_statements_dir, "task_statements_2004_dec.csv"),  
    2005: os.path.join(onet_task_statements_dir, "task_statements_2005_dec.csv"),  
    2006: os.path.join(onet_task_statements_dir, "task_statements_2006_dec.csv"),  
    2007: os.path.join(onet_task_statements_dir, "task_statements_2007_jun.csv")
}

task_statements_2003_to_2007 = read_onet_task_data(onet_task_files)
task_statements_2003_to_2007

,O*NET-SOC Code,Task ID,Task,Task Type,Incumbents Responding,Scale ID,Data Value,N,Standard Error,Lower CI Bound,...,Recommend Suppress-F6,Percent Frequency: Hourly Or More-F7,N-F7,Standard Error-F7,Lower CI Bound-F7,Upper CI Bound-F7,Recommend Suppress-F7,Date,Domain Source,year
0,11-1011.01,NaN,Directs organization charged with administerin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003
1,11-1011.01,NaN,"Administers, interprets, and explains policies...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003
2,11-1011.01,NaN,"Develops, plans, organizes, and administers po...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003
3,11-1011.01,NaN,Directs and coordinates activities of workers ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003
4,11-1011.01,NaN,Negotiates contracts and agreements with feder...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78104,53-7121.00,12806.0,"Test vessels for leaks, damage, and defects, a...",Supplemental,66.0,IM,4.04,24.0,0.23,3.57,...,N,0.93,23.0,0.83,0.14,5.73,N,12/2006,Incumbent,2007
78105,53-7121.00,12807.0,Unload cars containing liquids by connecting h...,Supplemental,66.0,IM,4.00,27.0,0.04,3.91,...,N,2.70,27.0,2.66,0.34,18.26,N,12/2006,Incumbent,2007
78106,53-7121.00,12808.0,Copy and attach load specifications to loaded ...,Supplemental,64.0,IM,3.99,23.0,0.11,3.77,...,N,7.31,22.0,6.37,1.11,35.75,N,12/2006,Incumbent,2007
78107,53-7121.00,12809.0,Start pumps and adjust valves or cables in ord...,Supplemental,67.0,IM,3.92,37.0,0.24,3.43,...,N,34.94,35.0,19.20,8.80,74.93,Y,12/2006,Incumbent,2007


In [58]:
# Construct the IM data
# Drop any rows with recommend suppress = 'Y' and with a null importance score
tr_2003_to_2007_IM = task_statements_2003_to_2007[
    (task_statements_2003_to_2007['Recommend Suppress'] == 'N') &
    (task_statements_2003_to_2007['Data Value'].notna()) & 
    (task_statements_2003_to_2007['Scale ID'] == 'IM')
].copy()

# Construct the FT data 
tr_2003_to_2007_FT = task_statements_2003_to_2007[
    (task_statements_2003_to_2007['Recommend Suppress-F1'] == 'N') &
    (task_statements_2003_to_2007['Recommend Suppress-F2'] == 'N') &
    (task_statements_2003_to_2007['Recommend Suppress-F3'] == 'N') &
    (task_statements_2003_to_2007['Recommend Suppress-F4'] == 'N') &
    (task_statements_2003_to_2007['Recommend Suppress-F5'] == 'N') &
    (task_statements_2003_to_2007['Recommend Suppress-F6'] == 'N') &
    (task_statements_2003_to_2007['Recommend Suppress-F7'] == 'N') &
    (task_statements_2003_to_2007['Percent Frequency: Yearly Or Less-F1'].notna()) &
    (task_statements_2003_to_2007['Percent Frequency: More Than Yearly-F2'].notna()) &
    (task_statements_2003_to_2007['Percent Frequency: More Than Monthly-F3'].notna()) &
    (task_statements_2003_to_2007['Percent Frequency: More Than Weekly-F4'].notna()) &
    (task_statements_2003_to_2007['Percent Frequency: Daily-F5'].notna()) &
    (task_statements_2003_to_2007['Percent Frequency: Several Times Daily-F6'].notna()) &
    (task_statements_2003_to_2007['Percent Frequency: Hourly Or More-F7'].notna())
].copy()

In [63]:
# Group by 'O*NET-SOC Code' and 'year', and aggregate 'Data Value' as:
# - total sum over all rows
# - sum only for rows where 'Task Type' == 'Core'
tr_2003_to_2007_IM_grp = tr_2003_to_2007_IM.groupby(['O*NET-SOC Code', 'year']).agg(
    Data_Value_sum_all=('Data Value', 'sum'),
    Data_Value_sum_core=('Data Value', lambda x: x[tr_2003_to_2007_IM.loc[x.index, 'Task Type'] == 'Core'].sum())
).reset_index()

# Merge the aggregated values back into the original DataFrame
tr_2003_to_2007_IM = pd.merge(tr_2003_to_2007_IM, tr_2003_to_2007_IM_grp, on=['O*NET-SOC Code', 'year'], how='left')
# Normalize the 'Data Value' by the total sum and core sum
tr_2003_to_2007_IM['IM_normalized'] = tr_2003_to_2007_IM['Data Value'] / tr_2003_to_2007_IM['Data_Value_sum_all']
tr_2003_to_2007_IM['IM_normalized_core_only'] = tr_2003_to_2007_IM['Data Value'] / tr_2003_to_2007_IM['Data_Value_sum_core']


# Calculate the expected frequency for each task
tr_2003_to_2007_FT['expected_frequency'] = (tr_2003_to_2007_FT['Percent Frequency: Yearly Or Less-F1'] * 1 +
                                              tr_2003_to_2007_FT['Percent Frequency: More Than Yearly-F2'] * 2 +
                                              tr_2003_to_2007_FT['Percent Frequency: More Than Monthly-F3'] * 3 +
                                              tr_2003_to_2007_FT['Percent Frequency: More Than Weekly-F4'] * 4 +
                                              tr_2003_to_2007_FT['Percent Frequency: Daily-F5'] * 5 +
                                              tr_2003_to_2007_FT['Percent Frequency: Several Times Daily-F6'] * 6 +
                                              tr_2003_to_2007_FT['Percent Frequency: Hourly Or More-F7'] * 7) / 100

# Merge back in
task_statements_2003_to_2007_final = pd.merge(task_statements_2003_to_2007, tr_2003_to_2007_IM[['O*NET-SOC Code', 'Task ID', 'year', 'IM_normalized', 'IM_normalized_core_only']],
                                        on=['O*NET-SOC Code', 'year', 'Task ID'], how='left')
task_statements_2003_to_2007_final = pd.merge(task_statements_2003_to_2007_final, tr_2003_to_2007_FT[['O*NET-SOC Code', 'Task ID', 'year', 'expected_frequency']],
                                        on=['O*NET-SOC Code', 'year', 'Task ID'], how='left')

task_statements_2003_to_2007_final

,O*NET-SOC Code,Task ID,Task,Task Type,Incumbents Responding,Scale ID,Data Value,N,Standard Error,Lower CI Bound,...,Standard Error-F7,Lower CI Bound-F7,Upper CI Bound-F7,Recommend Suppress-F7,Date,Domain Source,year,IM_normalized,IM_normalized_core_only,expected_frequency
0,11-1011.01,NaN,Directs organization charged with administerin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003,NaN,NaN,NaN
1,11-1011.01,NaN,"Administers, interprets, and explains policies...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003,NaN,NaN,NaN
2,11-1011.01,NaN,"Develops, plans, organizes, and administers po...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003,NaN,NaN,NaN
3,11-1011.01,NaN,Directs and coordinates activities of workers ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003,NaN,NaN,NaN
4,11-1011.01,NaN,Negotiates contracts and agreements with feder...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3/2002,Legacy Analyst,2003,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78104,53-7121.00,12806.0,"Test vessels for leaks, damage, and defects, a...",Supplemental,66.0,IM,4.04,24.0,0.23,3.57,...,0.83,0.14,5.73,N,12/2006,Incumbent,2007,0.051715,0.138641,NaN
78105,53-7121.00,12807.0,Unload cars containing liquids by connecting h...,Supplemental,66.0,IM,4.00,27.0,0.04,3.91,...,2.66,0.34,18.26,N,12/2006,Incumbent,2007,0.051203,0.137268,4.4290
78106,53-7121.00,12808.0,Copy and attach load specifications to loaded ...,Supplemental,64.0,IM,3.99,23.0,0.11,3.77,...,6.37,1.11,35.75,N,12/2006,Incumbent,2007,0.051075,0.136925,5.0694
78107,53-7121.00,12809.0,Start pumps and adjust valves or cables in ord...,Supplemental,67.0,IM,3.92,37.0,0.24,3.43,...,19.20,8.80,74.93,Y,12/2006,Incumbent,2007,0.050179,0.134523,NaN


In [82]:
task_statements_2003_to_2007_final_to_merge = task_statements_2003_to_2007_final[['O*NET-SOC Code', 'Task ID', 'year', 'Date', 'Data Value', 'IM_normalized', 'expected_frequency']].copy()
task_statements_2003_to_2007_final_to_merge.columns = ['O*NET-SOC Code', 'Task ID', 'year', 'Date', 'Mean Importance', 'Importance Normalized All', 'Mean Frequency']
task_statements_2003_to_2007_final_to_merge

,O*NET-SOC Code,Task ID,year,Date,Mean Importance,Importance Normalized All,Mean Frequency
0,11-1011.01,NaN,2003,3/2002,NaN,NaN,NaN
1,11-1011.01,NaN,2003,3/2002,NaN,NaN,NaN
2,11-1011.01,NaN,2003,3/2002,NaN,NaN,NaN
3,11-1011.01,NaN,2003,3/2002,NaN,NaN,NaN
4,11-1011.01,NaN,2003,3/2002,NaN,NaN,NaN
...,...,...,...,...,...,...,...
78104,53-7121.00,12806.0,2007,12/2006,4.04,0.051715,NaN
78105,53-7121.00,12807.0,2007,12/2006,4.00,0.051203,4.4290
78106,53-7121.00,12808.0,2007,12/2006,3.99,0.051075,5.0694
78107,53-7121.00,12809.0,2007,12/2006,3.92,0.050179,NaN


In [86]:
# merge the 2008 onwards data with the 2003 to 2007 data
task_ratings_final = pd.concat([tr_2008_onwards_final[['O*NET-SOC Code', 'Task ID', 'year', 'Date', 'Mean Importance', 'Importance Normalized All', 'Mean Frequency']], task_statements_2003_to_2007_final_to_merge], ignore_index=True)
task_ratings_final

,O*NET-SOC Code,Task ID,year,Date,Mean Importance,Importance Normalized All,Mean Frequency
0,11-1011.00,8823.0,2008,06/2006,4.51,0.037502,3.6369
1,11-1011.00,8823.0,2009,06/2006,4.51,0.037502,3.6369
2,11-1011.00,8823.0,2010,06/2006,4.51,0.037502,3.6369
3,11-1011.00,8823.0,2011,06/2006,4.51,0.037502,3.6369
4,11-1011.00,8823.0,2012,06/2006,4.51,0.037502,3.6369
...,...,...,...,...,...,...,...
328698,53-7121.00,12806.0,2007,12/2006,4.04,0.051715,NaN
328699,53-7121.00,12807.0,2007,12/2006,4.00,0.051203,4.4290
328700,53-7121.00,12808.0,2007,12/2006,3.99,0.051075,5.0694
328701,53-7121.00,12809.0,2007,12/2006,3.92,0.050179,NaN


In [87]:
# save the final task ratings data to a CSV file
task_ratings_final.to_csv('/Users/sidsatya/dev/ailabor/onet_transformations/intermediate_data/task_ratings.csv', index=False)